# 신경망 활용 처음부터 끝까지: 분류와 회귀

## 주요 내용

- 이진 분류 신경망 모델: 영화 후기 분류

- 다중 클래스 분류 신경망 모델: 뉴스 기사 분류

- 회귀 신경망 모델: 주택 가격 예측

### 머신러닝 주요 용어

| 한글 | 영어 | 뜻 |
| :--- | :--- | :--- |
| 샘플, 입력값 | sample, input | 모델 훈련에 사용되는 데이터 |
| 예측값, 출력값 | prediction, output | 모델이 계산한 예측값 |
| 타깃 | target | 모델이 맞춰야 하는 값 |
| 손실값, 비용, 예측 오차 | loss value | 타깃과 예측값 사이의 오차. 문제 유형에 따라 측정법 다름. |
| 손실 함수, 비용 함수| loss function | 손실값(비용)을 계산하는 함수. |
| 클래스 | class | 분류 모델에서 각각의 샘플이 속하는 범주(클래스) |
| 라벨 | label | 분류 모델에서 타깃 대신 사용하는 표현 |
| 이진 분류 | binary classification | 양성/음성, 긍정/부정 등 샘플을 두 개의 클래스로 분류. |
| 다중 클래스 분류 | multiclass classification | 샘플을 세 개 이상의 클래스로 분류. 손글씨 숫자 분류 등. |
| 다중 라벨 분류 | multilabel classification | 샘플에 대해 두 종류 이상의 라벨을 지정하는 분류. 한 장의 사진에 강아지, 고양이, 토끼 등 여러 종의 포함 여부 확인. 각각의 종에 대해 범부를 맞춰야 함. |
| (스칼라) 회귀 | (scalar) regression | 샘플 별로 하나의 값만 예측하기. 주택 가격 예측 등. |
| 벡터 회귀 | vector regression | 샘플 별로 두 종류 이상의 값 예측하기. 네모 상자의 좌표 등. |
| (미니)배치 | mini-batch/batch | 보통 16, 32, 64, 128 등의 개수의 샘플로 구성된 묶음(배치). 훈련 루프의 스텝에 사용되는 훈련셋. |

## 영화 후기: 이진 분류

영화 후기가 긍정적인지 부정적인지를 판단하는 이진 분류 모델을 구성한다.

### 데이터 준비: IMDB 데이터셋

- 긍정 후기와 부정 후기 각각 25,000개

- [IMDB(Internet Moview Database)](https://www.imdb.com/) 영화 후기 사이트

### 케라스 데이터셋 모듈

[`tf.keras.datasets` 모듈](https://keras.io/api/datasets/)이 몇 개의 연습용 데이터셋을 제공한다.

- MNIST 손글씨 숫자 분류 데이터셋
- CIFAR10 작은 이미지 분류 데이터셋
- CIFAR100 작은 이미지 분류 데이터셋
- IMDB 영화 후기 감성 분류 데이터셋
- Reuters 단문 기사 주제 분류 데이터셋
- 패션 MNIST(Fashion MNIST) dataset
- 보스턴 주택 가격(Boston Housing price) 회귀 데이터셋

### 케라스 데이터셋의 `load_data()` 함수

각 데이터셋의 `load_data()` 메서드를 활용하여 데이터셋을 불러온다.

```python
>>> from tensorflow.keras.datasets import imdb

>>> (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
```

단어 사용 빈도가 높은 10,000개 단어만 사용한다.

- 그 이외에는 사용 빈도가 너무 낮아 모델 훈련에 도움되지 않는다.
- `num_words=10000` 키워드 인자를 활용한다.

### 데이터 살펴보기

후기 샘플 하나에 사용되는 단어의 수는 일정하지 않다.

```python
>>> len(train_data[0])
218

>>> len(train_data[1])
189
```

각각의 정수는 특정 단어를 가리킨다.

```python
>>> train_data[0][:10]
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]
```

훈련셋 0번 샘플은 긍정 후기를, 테스트셋의 0번 샘플은 부정 후기를 가리킨다.

```python
>>> train_labels[0]
1
>>> test_labels[0]
0
```

### 데이터 전처리: 벡터화와 멀티-핫 인코딩

- **벡터화**<font size='2'>vectorization</font>
    - 가장 긴 길이의 샘플 길이를 확인한다.
    - 확인된 길이에 맞춰 모든 샘플을 확장한다.
    - 확장에 사용되는 값은 기존 샘플에 사용되지 않은 값을 사용한다.
    - 예를 들어 여백을 의미하는 0을 사용할 수 있다.

- **멀티-핫 인코딩**<font size='2'>multi-hot encoding</font>
    - 0과 1로만 이루어진 일정한 길이의 벡터(1차원 어레이)로 변환한다.

### 영화 후기 멀티-핫 인코딩

- 어레이 길이: 10,000
- 항목: 0 또는 1
- 후기 샘플에 포함된 정수에 해당하는 인덱스의 항목만 1로 지정

예를 들어, `[1, 5, 9998]` 변환하기:

- 길이가 10,000인 1차원 어레이(벡터)로 변환
- 1번, 5번, 9998번 인덱스의 항목만 1이고 나머지는 0

    ```
    멀티-핫-인코딩([1, 5, 9998]) => [0, 1, 0, 0, 0, 1, 0, ..., 0, 0, 1, 0]
    ```

```python
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    
    for i, seq in enumerate(sequences):
        for j in seq:
            results[i, j] = 1.
    return results
```

```python
>>> x_train = vectorize_sequences(train_data).astype("float32")

>>> x_test = vectorize_sequences(test_data).astype("float32")
```

라벨(타깃)은 멀티-핫 인코딩을 적용하지 않는다.
다만 입력 샘플의 자료형과 맞추기 위해 `float32` 자료형으로 변환한다.

```python
>>> y_train = np.asarray(train_labels).astype("float32")

>>> y_train
array([1., 0., 0., ..., 0., 1., 0.], dtype=float32)
```

### 모델 구성

- 입력 샘플이 벡터(1차원 어레이)로 주어지고 라벨이 스칼라(하나의 숫자)일 때: 
    - 밀집층<font size='2'>densely-connected layer</font>인 `Dense` 층
    - `Sequential` 모델 이용 추천

- 은닉층의 활성화 함수: `relu`, `prelu`, `elu`, `tanh` 등이 많이 사용됨. 일반적으로 `relu` 추천.

- 이진 분류 모델의 최상위 출력층의 활성화 함수: 0과 1사이의 확률값을 계삲하는 `sigmoid` 함수

- 다중 클래스 분류 모델의 최상위 출력층의 활성화 함수: 클래스별 확률값을 계산하는 `softmax` 함수

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/dlp/master/slides/images/relu_sigmoid.png" style="width:600px;"></div>

### Dense 층 활용

- 몇 개의 층을 사용하는가?

- 각 층마다 몇 개의 유닛<font size='2'>unit</font>을 사용하는가?

```python
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
    ])
```

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/04-01.png" style="width:200px;"></div>

### 이진 분류 모델 컴파일

```python
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"]
             )
```

### 모델 훈련과 활용

훈련 중인 모델을 에포크마다 검증하기 위해 검증셋을 따로 지정한다.

```python
x_val = x_train[:10000]            # 검증용
partial_x_train = x_train[10000:]  # 훈련용
y_val = y_train[:10000]            # 검증용 타깃셋
partial_y_train = y_train[10000:]  # 훈련용 타깃셋

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val) # 검증 데이터셋 지정
                   )
```

### `fit()` 메서드 반환값: `History` 객체

```python
>>> history_dict = history.history
>>> history_dict.keys()
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
```

### 손실값과 정확도의 변화

<div align="center">
    <table>
        <tr>
            <td><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/04-04.png" style="width:500px;"></td>
            <td><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/04-05.png" style="width:500px;"></td>
        </tr>
    </table>

</div>

### 과대적합 방지

- **과대적합**<font size='2'>overfitting</font>: 모델이 훈련셋에 익숙해져서 처음 보는 데이터에 대해서 성능이 더 이상 좋아지지 않거나 떨어지는 현상

- 이전 모델: 4번째 에포크 이후로 과대적합 발생

- 4번의 에포크만 훈련 반복을 진행하면 과대적합되지 않은 모델이 훈련됨

- 모델 재훈련
    - 모델 구성부터, 컴파일, 훈련을 모두 처음부터 다시 시작
    - 가중치와 편향이 초기화된 상태로 훈련이 다시 시작됨

```python
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.fit(x_train, y_train, epochs=4, batch_size=512)
```

### 훈련 결과 테스트

```python
>>> results = model.evaluate(x_test, y_test)
>>> results
[0.3139097988605499, 0.8770800232887268]
```

### 모델 활용

```python
>>> model.predict(x_test, batch_size=512)
array([[0.25440323],
       [0.9999424 ],
       [0.95840394],
       ...,
       [0.17153329],
       [0.10725482],
       [0.6672551 ]], dtype=float32)
```

## 뉴스 기사: 다중 클래스 분류

로이터<font size='2'>Reuter</font> 통신사가 1986년에 작성한 단문 기사를 주제별로 분류한다.

### 데이터 준비: 로이터 데이터셋

- 총 11,228개의 단문 기사
    - 훈련셋 크기: 8,982
    - 테스트셋 크기: 2,246

- 기사 주제: 총 46 개

- 각각의 기사는 하나의 주제와 연관됨.

- **다중 클래스 분류**<font size='2'>multiclass classification</font> 모델 훈련

```python
from tensorflow.keras.datasets import reuters
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)
```

### 데이터 살펴보기

각 샘플은 정수들의 리스트이다.

```python
>>> train_data[10]
[1, 245, 273, 207, 156, 53, 74, 160, 26, 14, 46, 296, 26, 39, 74, 2979,
3554, 14, 46, 4689, 4329, 86, 61, 3499, 4795, 14, 61, 451, 4329, 17, 12]
```

각 샘플에 대한 라벨은 0부터 45까지의 정수로 표현된다.
3번 주제는 소득(earn)을 가리킨다.

```python
>>> train_labels[10]
3
```

### 로이터 기사 주제

| 번호 | 주제 | 번호 | 주제 | 번호 | 주제 | 번호 | 주제 |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 0 | cocoa | 1 | grain| 2 | veg-oil | 3 | earn |
| 4 | acq | 5 | wheat | 6 | copper | 7 | housing |
| 8 | money-supply | 9 | coffee | 10 | sugar | 11 | trade |
| 12 | reserves | 13 | ship | 14 | cotton | 15 | carcass |
| 16 | crude | 17 | nat-gas | 18 | cpi | 19 | money-fx |
| 20 | interest | 21 | gnp | 22 | meal-feed | 23 | alum |
| 24 | oilseed | 25 | gold | 26 | tin | 27 | strategic-metal |
| 28 | livestock | 29 | retail | 30 | ipi | 31 | iron-steel |
| 32 | rubber | 33 | heat | 34 | jobs | 35 | lei |
| 36 | bop | 37 | zinc | 38 | orange | 39 | pet-chem |
| 40 | dlr | 41 | gas | 42 | silver | 43 | wpi |
| 44 | hog | 45 | lead | | | | |

### 입력 데이터셋 벡터화: 멀티-핫 인코딩

IMDB의 경우와 동일한 방식

```python
>>> x_train = vectorize_sequences(train_data)

>>> x_test = vectorize_sequences(test_data)
```

### 라벨 데이터셋 벡터화: 원-핫 인코딩

- 라벨은 0부터 45 사이의 값이다.

- **원-핫 인코딩**<font size='2'>one-hot encoding</font> 적용

예를 들어, 정수 3은 길이가 46인 벡터로 변환되는데 3번 인덱스에서만 1이고 나머지 항목은 모두 0이다.

```python
array([0, 0, 0, 1, 0, 0, ...., 0])
```

케라스의 `to_categorical()` 함수가 원-핫 인코딩을 지원한다.

```python
>>> from tensorflow.keras.utils import to_categorical
>>> y_train = to_categorical(train_labels)
>>> y_test = to_categorical(test_labels)
>>> y_train[0]
array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)
```

### 모델 구성

- 은닉층: 64개의 유닛 사용.
    - 이진 분류보다 훨씬 많은 46개의 클래스로 분류하기 위해 보다 많은 정보 필요

- 다중 클래스 분류 모델의 출력층: 클래스 수 만큼의 값으로 구성된 벡터를 출력하도록 여러 개의 유닛을 사용하는 `Dense` 밀집층을 사용
    - 활성화 함수: 모든 유닛에 대한 확률값의 합이 1이 되도록 하는 `softmax()`

```python
model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(46, activation="softmax")
])
```

### 모델 컴파일

```python
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
```

### 모델 훈련과 활용

```python
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = y_train[:1000]
partial_y_train = y_train[1000:]

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))
```

### 손실값과 정확도의 변화

<div align="center">
    <table>
        <tr>
            <td><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/04-06.png" style="width:500px;"></td>
            <td><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/04-07.png" style="width:500px;"></td>
        </tr>
    </table>

</div>

### 모델 재훈련

9번 에포크를 지나면서 과대적합이 발생

```python
model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(46, activation="softmax")
])
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.fit(x_train,
          y_train,
          epochs=9,
          batch_size=512)
```

### 모델 활용

훈련된 모델의 `predict()` 메서드를 이용한다.

```python
>>> predictions = model.predict(x_test)
>>> predictions[0].shape
(46,)
>>> np.sum(predictions[0])
1.0
```

테스트셋의 첫째 샘플에 대한 예측값은 3이다.

```python
>>> np.argmax(predictions[0])
3
```

## 주택가격: 회귀

이진 분류와 다중 클래스 분류는
몇 개의 클래스를 가리키는 숫자들 중에 하나를 예측하는 문제다.
반면에 임의의 실수를 예측하는 문제는 **회귀**<font size='2'>regression</font>라 부른다. 
예를 들어 온도 예측, 가격 예측을 하는 머신러닝 모델이 회귀 모델이다.

여기서는 미국 보스턴<font size='2'>Boston</font> 시의 1970년대 중반의 
주택가격을 예측하는 회귀 문제를 예제로 다룬다.

:::{admonition} 로지스틱 회귀
:class: warning

로지스틱 회귀<font size='2'>logistic regression</font> 알고리즘는 분류 모델임에 주의하라.
:::

### 데이터 준비: 보스턴 주택가격 데이터셋

사용하는 데이터셋은
1970년대 중반의 미국 보스턴 시 외곽의 총 506개 지역에서 수집된 통계 자료를 담고 있다.
통계는  지역별 중간 주택가격과 함께 다음 13가지 내용을 조사했다.

| 특성 | 의미 |
|:------|:---------|
| CRIM  | 구역별 1인당 범죄율 |
| ZN    | 25,000 평방 피트 이상의 주거 구역 비율 |
| INDUS | 구역별 비 소매 사업 에이커(acre) 비율 |
| CHAS  | Charles River 더미 변수(구역이 강 경계에 닿으면 1, 아니면 0) |
| NOX   | 산화 질소 농도(1000만분 율) |
| RM    | 주택 당 평균 방 수 |
| AGE   | 소유주가 살고 있는 1940년 이전에 지어진 건물 비율 |
| DIS   | 보스턴 고용 센터 다섯 곳 까지의 가중 거리 |
| RAD   | 방사형 고속도로 접근성 지수 |
| TAX   | 1만달러당 전체 가지 재산 세율 |
| PTRATIO | 구역별 학생-교사 비율 |
| B     | 1000(Bk - 0.63)^2 (Bk 구역별 흑인 비율) |
| LSTAT | 구역별 낮은 지위 인구 비율 |

언급된 13가지 데이터가 주어졌을 때 해당 구역의 중간 주택가격을 예측하는 회귀 모델을
훈련시켜야 한다.

:::{admonition} 보스턴 데이터셋의 윤리 문제
:class: hint

구역별로 조사된 자료 중에서 흑인 비율을 사용하는 `B` 특성이 윤리적 논쟁을 일으킨다.
구역의 집값과 흑인 비율의 연관성을 암시하는 이런 통계 조사는
1970년대 미국에서 인종 차별이 여전히 주요 쟁점이었음을 단편적으로 보여준다.
이런 이유로 사이킷런 라이브러리는
[`sklearn.datasets.load_boston()` 함수](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html)를 이용하여 제공하던 보스턴 데이터셋을
삭제하기로 예고했다.
여기서는 단순히 데이터 활용 차원에서만 보스턴 데이터셋을 이용할 뿐 다른 어떤 의도도 없음을 밝힌다.
:::

케라스 `boston_housing` 모듈의 `load_data()` 함수로 보스턴 데이터셋을 불러올 수 있다.
데이터셋이 이미 404개 샘플로 구성된 훈련셋과 102개 샘플로 구성된 테스트셋으로 구분되어 있다.

```python
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()
```

훈련셋과 테스트셋의 타깃 텐서는 아래처럼 범위가 지정되지 않은 부동소수점으로 구성된다.

```python
>>> train_targets
[ 15.2,  42.3,  50. ...  19.4,  19.4,  29.1]
```

### 데이터 전처리: 표준화

특성에 따라 사용되는 값들의 크기 정도<font size='2'>scale</font>가 다르다. 
어떤 특성은 0과 1사이의 값을, 다른 특성은 100단위의 값을 포함하기도 한다.
그런데 머신러닝 모델은 기본적으로 모든 특성이 동일한 크기 정도의 
값으로 구성될 때 보다 잘 훈련된다.
이런 이유로 여기서는 평균은 0, 표준편차는 1이 되도록 변환하는
**표준화**<font size='2'>standardization</font>를 적용해서
훈련셋과 테스트셋을 전처리한다.

표준화는 다음 식으로 계산된다. 

$$
\frac{x - \mu}{\sigma}
$$

$x$는 샘플의 특성값을, $\mu$와 $\sigma$는 훈련셋에 포함된 샘플들의 
특성별 평균값과 표준편차를 가리킨다.
넘파이 어레이를 이용하면 전체 훈련셋에 대해 한 번에 다음과 같이 표준화를 진행할 수 있다.

```python
# 훈련셋의 평균값
mean = train_data.mean(axis=0)

# 훈련셋 표준화
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

# 테스트셋 표준화: 훈련셋의 평균값과 표준편차 활용
test_data -= mean
test_data /= std
```

:::{admonition} 테스트셋 표준화
:class: warning

테스트셋의 표준화도 훈련셋의 평균값과 표준편차를 이용한다.
이유는 테스트셋의 정보는 모델 훈련에 절대로 사용되지 않아야 하기 때문이다.
:::

### 모델 구성과 컴파일

데이터셋이 작으므로 출력층을 제외하고 두 개 층만 사용한다.
머신러닝 모델은 훈련셋이 작을 수록 과대적합을 보다 잘하기 때문이
이를 방지하기 위해 보다 단순한 모델을 사용한다.

마지막 층을 제외한 나머지 층은 64개의 유닛과 함께 `relu()` 활성화 함수를 사용한다.
회귀 모델의 마지막 층은 기본적으로 활성화 함수 없이 1개의 유닛만 사용한다.
이유는 하나의 값을 예측하되 그 값의 크기를 제한하지 않아야 하기 때문이다.

모델 컴파일에 필요한 손실 함수와 평가지표는 다음과 같다.

- 손실함수: **평균제곱오차**<font size='2'>mean squared error</font>(mse). 
    타깃과 예측값 사이의 오차의 제곱의 평균값. 회귀 모델의 일반적인 손실 함수.
- 평가지표: **평균절대오차**<font size='2'>mean absolute error</font>(mae).
    타깃과 예측값 사이의 오차의 평균값. 

동일한 모델을 앞으로도 계속해서 사용할 예정이기에
여기서는 모델 구성과 컴파일을 동시에 진행하는 
하나의 함수를 지정해서 사용한다.

```python
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(1)
    ])
    model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
    return model
```

### 모델 훈련과 활용

데이터셋이 작기에 훈련 중에 사용할 검증 세트를 따로 분리하는 것은 훈련의 효율성을 떨어뜨린다.
대신에 **K-겹 교차검증**<font size='2'>K-fold cross-validation</font>을 사용한다.
아래 이미지는 3-겹 교차검증을 사용할 때 훈련 중에 사용되는 훈련셋과 검증셋의 사용법을 보여준다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/3-fold-cross-validation.png" style="width:600px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

아래 코드는 4-겹 교차검증을 구현한다.

훈련셋을 4등분 한 다음에 폴드별로 차례대로 검증셋으로 활용하여 모델을 4번 훈련시킨다.
훈련되는 모델은 앞서 `build_model()` 함수로 선언된 모델이며
폴드가 정해지면 매번 새롭게 선언된다.

```python
# 폴드 수
k = 4
# 검증 폴드의 크기
num_val_samples = len(train_data) // k
# 모델 훈련 에포크 수
num_epochs = 500
# 폴드별 평가지표 저장
all_mae_histories = []

# k-겹 교차검증
for i in range(k):
    print(f"{i+1}번 째 폴드(fold) 훈련 시작")
    # 검증 폴드 지정
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    # 훈련셋과 훈련 타깃 지정: 검증 폴드 이외 나머지 3개의 폴드
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    # 모델 지정
    model = build_model()
    # 모델 훈련
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=16, verbose=0)
    # 폴드별 평가지표 저장
    mae_history = history.history["val_mae"]
    all_mae_histories.append(mae_history)
```

4개의 폴드에 대해 매번 다른 폴드를 검증셋으로 지정하고 
모델 훈련을 500번 에포크 동안 진행하였다.
에포크별로 검증셋을 대상으로하는 평균절대오차(MAE)의 평균값을 계산하면
에포크별 MAE의 변화를 그래프로 확인할 수 있다.

```python
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]
```

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/04-10.png" style="width:600px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

:::{prf:example} 사이킷런의 `KFold` 클래스
:label: exp-k-fold

사이킷런의 `KFold` 클래스를 이용하면 봅다 간단하게 K-겹 교차검증을 진행할 수 있다.

```python
from sklearn.model_selection import KFold

k = 4
num_epochs = 500

kf = KFold(n_splits=k)
all_mae_histories = []

for train_index, val_index in kf.split(train_data, train_targets):
    
    val_data, val_targets = train_data[val_index], train_targets[val_index]
    partial_train_data, partial_train_targets = train_data[train_index], train_targets[train_index]
    
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=16, verbose=0)

    mae_history = history.history["val_mae"]    
    all_mae_histories.append(mae_history)
```
:::

**모델 재훈련**

130번 째 에포크를 전후로 과대적합이 발생함을 확인할 수 있다.
따라서 130번의 에포크만 사용해서 모델을 재훈련 하면 좋은 성능의 모델을 얻는다.

```python
model = build_model()
model.fit(train_data, train_targets,
          epochs=130, batch_size=16, verbose=0)
```

재훈련된 모델의 테스트셋에 대한 성능을 평가하면 
주택가격 예측에 있어서 평균적으로 2,500달러 정도의 차이를 갖는다.

```python
>>> test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
>>> test_mae_score
2.4642276763916016
```

**모델 활용**

새로운 데이터에 대한 예측은 `predict()` 메서드를 활용한다.
집갑을 예측하는 모델이기에 하나의 수가 예측된다.

```python
>>> predictions = model.predict(test_data)
>>> predictions[0]
array([9.990133], dtype=float32)
```

## 연습문제

1. [(실습) 신경망 활용 처음부터 끝까지: 분류와 회귀](https://colab.research.google.com/github/codingalzi/dlp2/blob/master/excs/exc-getting_started_with_neural_networks.ipynb)